In [365]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium
from folium import plugins
from folium.plugins import HeatMap

In [383]:
hood = pd.read_csv('./toronto-copy.csv')

In [384]:
#hood = pd.read_csv('/Users/adarsh/Downloads/toronto-copy.csv')topics_and_chars = set(zip(hood["Topic"].apply(lambda x : x.strip()), hood["Characteristic"].apply(lambda x : x.strip())))
d = {}
for t, c in topics_and_chars:
    if t not in d: d[t] = []
    d[t].append(c)
for t in d:
    if "ethnic" in t.lower():
        print(t, sorted(d[t]))
        
middle_eastern = ['Afghan', 'Arab', 'Egyptian', 'Saudi', 'Iran', 'Iraq', 'Turk', 'Lebanese', 'Greek']
mid_east_hoods = []
is_middle_eastern = lambda s: any(c.lower() in s.lower() for c in middle_eastern)
ret = None
for index, row in hood.iterrows():
    if "ethnic origin" in row["Topic"].lower():
        if is_middle_eastern(row["Characteristic"]):
            by_hood = row[5:].apply(lambda a: int(a.replace(',', '')))
            if ret is None:
                ret = by_hood
            else:
                ret = ret + by_hood


Ethnic origin population ['Aboriginal from Central/South America (except Arawak and Maya)', 'Acadian', 'Afghan', 'African origins', 'Afrikaner', 'Akan', 'Albanian', 'Algerian', 'Alsatian', 'American', 'Amhara', 'Angolan', 'Antiguan', 'Arab; n.o.s.', 'Arawak', 'Argentinian', 'Armenian', 'Ashanti', 'Asian origins', 'Assyrian', 'Australian', 'Austrian', 'Azerbaijani', 'Bahamian', 'Bangladeshi', 'Bantu; n.o.s.', 'Barbadian', 'Basque', 'Bavarian', 'Belgian', 'Belizean', 'Bengali', 'Beninese', 'Berber', 'Bermudan', 'Bhutanese', 'Black; n.o.s.', 'Bolivian', 'Bosnian', 'Brazilian', 'Breton', 'British Isles origins', 'British Isles origins; n.i.e.', 'Bulgarian', 'Burkinabe', 'Burmese', 'Burundian', 'Byelorussian', 'Cambodian (Khmer)', 'Cameroonian', 'Canadian', 'Carib', 'Caribbean origins', 'Caribbean origins; n.i.e.', 'Catalan', 'Central and West African origins', 'Central and West African origins; n.i.e.', 'Chadian', 'Channel Islander', 'Chilean', 'Chinese', 'Colombian', 'Congolese', 'Coptic'

In [385]:
pop = 'Population, 2016'
hood_pop = hood.loc[hood['Characteristic'] == pop]
hood_pop = hood_pop.iloc[0][6:].apply(lambda x: int(x.replace(",", "")))
hood_pop = hood_pop.to_frame()
hood_pop.columns = ['Total Pop']
hood_pop

,Total Pop
Agincourt North,29113
Agincourt South-Malvern West,23757
Alderwood,12054
Annex,30526
Banbury-Don Mills,27695
...,...
Wychwood,14349
Yonge-Eglinton,11817
Yonge-St.Clair,12528
York University Heights,27593


In [386]:
ret = ret[1:]
ret

Agincourt North                  645
Agincourt South-Malvern West    1150
Alderwood                        390
Annex                           1850
Banbury-Don Mills               2765
                                ... 
Wychwood                         780
Yonge-Eglinton                   775
Yonge-St.Clair                   620
York University Heights         1770
Yorkdale-Glen Park               470
Length: 140, dtype: int64

In [387]:
# ret.columns = ['Total']
# ret_desc = ret.sort_values(['Total'], ascending=[False])
# ret_desc.columns = ['AREA_NAME', 'Total']
# ret_desc = ret_desc.loc[ret_desc['AREA_NAME'] != 'City of Toronto']
# ret_desc['AREA_NAME'] = ret_desc['AREA_NAME'].apply(lambda x : x.strip())
# ret_desc.set_index('AREA_NAME', inplace=True)
# ret_desc
ret_desc = ret.to_frame()
ret_desc.columns = ['ME Population']
ret_desc

,ME Population
Agincourt North,645
Agincourt South-Malvern West,1150
Alderwood,390
Annex,1850
Banbury-Don Mills,2765
...,...
Wychwood,780
Yonge-Eglinton,775
Yonge-St.Clair,620
York University Heights,1770


In [371]:
hood_latlng = pd.read_csv('/Users/adarsh/Downloads/Neighbourhoods.csv')
print(hood_latlng.shape)
hood_latlng.head()
hood_latlng['AREA_NAME'] = hood_latlng['AREA_NAME'].apply(lambda name: name[:name.rindex("(")].strip())
hood_latlng.set_index('AREA_NAME', inplace=True)
hood_latlng

(140, 16)


,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
AREA_NAME,,,,,,,,,,,,,,,
Wychwood,8961,25886861,25926662,49885,94,94,Wychwood (94),NaN,NaN,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
Yonge-Eglinton,8962,25886820,25926663,49885,100,100,Yonge-Eglinton (100),NaN,NaN,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
Yonge-St.Clair,8963,25886834,25926664,49885,97,97,Yonge-St.Clair (97),NaN,NaN,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"{u'type': u'Polygon', u'coordinates': (((-79.3..."
York University Heights,8964,25886593,25926665,49885,27,27,York University Heights (27),NaN,NaN,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"{u'type': u'Polygon', u'coordinates': (((-79.5..."
Yorkdale-Glen Park,8965,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),NaN,NaN,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kennedy Park,9096,25886531,25926797,49885,124,124,Kennedy Park (124),NaN,NaN,-79.260382,43.725556,16493665,6.861056e+06,10530.202433,"{u'type': u'Polygon', u'coordinates': (((-79.2..."
Kensington-Chinatown,9097,25886955,25926798,49885,78,78,Kensington-Chinatown (78),NaN,NaN,-79.397240,43.653554,16493681,2.933586e+06,6945.056557,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
Kingsview Village-The Westway,9098,25886737,25926799,49885,6,6,Kingsview Village-The Westway (6),NaN,NaN,-79.547863,43.698993,16493697,9.694496e+06,14458.764271,"{u'type': u'Polygon', u'coordinates': (((-79.5..."


In [372]:
#df2 = df_latlng.merge(df, on='Postal Code')
hood_ret = ret_desc.join(hood_latlng).join(hood_pop)
hood_ret

,ME Population,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry,Total Pop
Agincourt North,645,9035,25886428,25926736,49885,129,129,Agincourt North (129),NaN,NaN,-79.266712,43.805441,16492689,1.395145e+07,17159.740667,"{u'type': u'Polygon', u'coordinates': (((-79.2...",29113
Agincourt South-Malvern West,1150,9036,25886449,25926737,49885,128,128,Agincourt South-Malvern West (128),NaN,NaN,-79.265612,43.788658,16492705,1.511736e+07,21320.849547,"{u'type': u'Polygon', u'coordinates': (((-79.2...",23757
Alderwood,390,9037,25886794,25926738,49885,20,20,Alderwood (20),NaN,NaN,-79.541611,43.604937,16492721,9.502180e+06,12667.013917,"{u'type': u'Polygon', u'coordinates': (((-79.5...",12054
Annex,1850,9038,25886874,25926739,49885,95,95,Annex (95),NaN,NaN,-79.404001,43.671585,16492737,5.337192e+06,10513.883143,"{u'type': u'Polygon', u'coordinates': (((-79.3...",30526
Banbury-Don Mills,2765,9039,25886643,25926740,49885,42,42,Banbury-Don Mills (42),NaN,NaN,-79.349718,43.737657,16492753,1.924897e+07,25141.572290,"{u'type': u'Polygon', u'coordinates': (((-79.3...",27695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wychwood,780,8961,25886861,25926662,49885,94,94,Wychwood (94),NaN,NaN,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"{u'type': u'Polygon', u'coordinates': (((-79.4...",14349
Yonge-Eglinton,775,8962,25886820,25926663,49885,100,100,Yonge-Eglinton (100),NaN,NaN,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"{u'type': u'Polygon', u'coordinates': (((-79.4...",11817
Yonge-St.Clair,620,8963,25886834,25926664,49885,97,97,Yonge-St.Clair (97),NaN,NaN,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"{u'type': u'Polygon', u'coordinates': (((-79.3...",12528
York University Heights,1770,8964,25886593,25926665,49885,27,27,York University Heights (27),NaN,NaN,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"{u'type': u'Polygon', u'coordinates': (((-79.5...",27593


In [388]:
hood_ret = hood_ret[['LONGITUDE', 'LATITUDE', 'Total Pop', 'ME Population']]
hood_ret.head()

,LONGITUDE,LATITUDE,Total Pop,ME Population
Agincourt North,-79.266712,43.805441,29113,645
Agincourt South-Malvern West,-79.265612,43.788658,23757,1150
Alderwood,-79.541611,43.604937,12054,390
Annex,-79.404001,43.671585,30526,1850
Banbury-Don Mills,-79.349718,43.737657,27695,2765


In [389]:
print(hood_ret.shape)
hood_ret.head()

(140, 4)


,LONGITUDE,LATITUDE,Total Pop,ME Population
Agincourt North,-79.266712,43.805441,29113,645
Agincourt South-Malvern West,-79.265612,43.788658,23757,1150
Alderwood,-79.541611,43.604937,12054,390
Annex,-79.404001,43.671585,30526,1850
Banbury-Don Mills,-79.349718,43.737657,27695,2765


In [391]:
test = hood_ret.sort_values(['ME Population'], ascending=[False])
test['quasi_pop'] = test['ME Population']/test['Total Pop']
test = test.sort_values(['quasi_pop'], ascending=[False])
test.head(10)


,LONGITUDE,LATITUDE,Total Pop,ME Population,quasi_pop
Newtonbrook East,-79.405937,43.791536,16097,3175,0.197242
Old East York,-79.335488,43.696781,9233,1755,0.190079
Willowdale East,-79.401484,43.770602,50434,8970,0.177856
Broadview North,-79.355630,43.688825,11499,2010,0.174798
Flemingdon Park,-79.332646,43.715930,21933,3755,0.171203
Bayview Village,-79.377117,43.776361,21396,3645,0.170359
Victoria Village,-79.314874,43.728489,17510,2880,0.164477
Henry Farm,-79.341241,43.771144,15723,2530,0.160911
Thorncliffe Park,-79.349984,43.707749,21108,3310,0.156813
Wexford/Maryvale,-79.298637,43.748572,27917,4360,0.156177


In [392]:
map_hood_ret = folium.Map(location=[43.6, -79.2], zoom_start=10)
hood_per_capita = hood_ret['ME Population'].divide(hood_ret['Total Pop'])
print(hood_per_capita)
hoodret_map = list(zip(hood_ret['LATITUDE'], hood_ret['LONGITUDE'], hood_per_capita))
plugins.HeatMap(hoodret_map, radius=7, blur=3).add_to(map_hood_ret)
map_hood_ret

Agincourt North                 0.022155
Agincourt South-Malvern West    0.048407
Alderwood                       0.032354
Annex                           0.060604
Banbury-Don Mills               0.099838
                                  ...   
Wychwood                        0.054359
Yonge-Eglinton                  0.065583
Yonge-St.Clair                  0.049489
York University Heights         0.064147
Yorkdale-Glen Park              0.031748
Length: 140, dtype: float64
